In [12]:
"""
Inputs: 
    trait sheet with traits for each NFT
    IPFS prefix for male and female images
    image file suffix

This can easily be modified to shift or shuffle the metadata using an integer seed

"""
import pandas as pd
import json
#import random #do we want to seed this?
#import numpy as np
import re
import os
#SET CONSTANTS
TRAIT_SHEET='shuffled_03_11_21_HonestNFT_Vigilante_Metadata.csv'
IPFS_PREFIX = 'ipfs://QmQPwK9RGVok31N7MEkZPhYnpn3G2SPQKcbFkcT2cJrDas/'
MASCULINE_SUFFIX = '/base_m.png'
FEMININE_SUFFIX = '/base_f.png'

UNREVEALED_IMAGE='ipfs://QmZEBPhxmnTTbKX1QEVLwPdSeRAQ15F6UydhmwT5eHDc1f/'

#token ID to replace revealed image with unrevealed placeholder
START_UNREVEALED = 1400
DESCRIPTION = '3,777 Vigilantes on a mission to make NFT launches fair and equitable. Join us. https://vigilante.eth.limo/'

curpath = os.path.abspath(os.curdir)
FILE_PATH = curpath+'/metadata'

#load traits

trait_sheet=pd.read_csv(TRAIT_SHEET)


def stdTrait(str):
    str = re.sub(r"(_|-)", " ", str)
    str = str.title()
    return str
def save_unrevealed(nft_metadata):
    attributes = []
    for attribute in ['Mask', 'Outfit', 'Eyes','Headgear','Accessory','Gender','Background']:
            #if it's a male NFT give male traits

            attributes.append({
                "trait_type": stdTrait(attribute),
                "value": 'Unrevealed'})
            
    metadata = {
    "name": 'Vigilante #' + str(nft_metadata['TOKEN ID']),
    "description": DESCRIPTION,
    "attributes": attributes}
    metadata['image']=UNREVEALED_IMAGE
    output_file=str(nft_metadata['TOKEN ID'])
    complete_name = os.path.join(FILE_PATH, output_file)
    with open(complete_name, 'w') as outfile:
        json.dump(metadata, outfile, indent=4)
                

# function to convert strings to json format and saves the file as .json
def save_metadata(nft_metadata):
    attributes = []
    #note 'Backgrounds' is being changed to 'Background' here
    masculine_trait_labels = {'Mask': 'Masks', 'Headgear': 'Headgear','Outfit':'Outfit','Accessory':'Accessory_M','Eyes':'Eyes','Background':'Background','Gender':'Gender'}
    feminine_trait_labels = {'Mask': 'Masks', 'Headgear': 'Headgear','Outfit':'Outfit','Accessory':'Accessory_F','Eyes':'Eyes','Background':'Background','Gender':'Gender'}

    for attribute in ['Mask', 'Outfit', 'Eyes','Headgear','Accessory','Gender','Background']:
            #if it's a masculine NFT give masc traits
            if nft_metadata['Gender']=='Masculine':
                attributes.append({
                    "trait_type": stdTrait(attribute),
                    "value": stdTrait(nft_metadata[masculine_trait_labels[attribute]])
                })
            else:#if it's a feminine NFT give female traits
                 attributes.append({
                    "trait_type": stdTrait(attribute),
                    "value": stdTrait(nft_metadata[feminine_trait_labels[attribute]])
                })               

    metadata = {
        "name": 'Vigilante #' + str(nft_metadata['TOKEN ID']),
        "description": DESCRIPTION,
        "attributes": attributes}

    if nft_metadata['Gender']=='Masculine':
        metadata['image']=IPFS_PREFIX+str(nft_metadata['Image_Number'])+MASCULINE_SUFFIX
    else:
        metadata['image']=IPFS_PREFIX+str(nft_metadata['Image_Number'])+FEMININE_SUFFIX
             
    output_file=str(nft_metadata['TOKEN ID'])
    complete_name = os.path.join(FILE_PATH, output_file)
    with open(complete_name, 'w') as outfile:
        json.dump(metadata, outfile, indent=4)

        
        
            
        


In [13]:

for nft in range(0,START_UNREVEALED):
    save_metadata(trait_sheet.loc[nft])
for nft in range(START_UNREVEALED,len(trait_sheet)):
    save_unrevealed(trait_sheet.loc[nft])
